# Natural Disasters: Revision
This is a revision of the original notebook "Natural Disasters" by Hannah.

In [18]:
import pandas as pd
# Read
raw = pd.read_csv(
    "inputs/disasters_emdat.csv",
    encoding="latin1",
    skiprows=6,
    usecols=[
        "Year",
        "Disaster Type",
        "Country",
        "No Homeless",
        "Total Deaths",
        "No Injured",
        "No Affected",
        "No Homeless",
        "Total Affected",
        "Reconstruction Costs ('000 US$)",
        "Insured Damages ('000 US$)",
        "Total Damages ('000 US$)"
    ],
)

In [19]:
# Column renaming
raw = raw.rename(
    columns={
        "Disaster Type": "disaster_type",
        "Total Deaths": "deaths",
        "No Injured": "injured",
        "No Affected": "affected",
        "No Homeless": "homeless",
        "Total Affected": "total_affected",
        "Reconstruction Costs ('000 US$)": "reconstruction_costs",
        "Insured Damages ('000 US$)": "insured_damages",
        "Total Damages ('000 US$)": "total_damages",
    }
)

In [20]:
# Get metrics
combined = raw.groupby(["Year", "disaster_type", "Country"], as_index=False)[[
    "deaths", 
    "injured", 
    "affected", 
    "homeless", 
    "total_affected", 
    "reconstruction_costs", 
    "insured_damages", 
    "total_damages"
]].sum()

In [21]:
# Standardize country names
countries = pd.read_csv("inputs/countries.csv", encoding="latin1")
combined = combined.merge(countries, on="Country", how="left")
print("Countries without standardisation:", combined.loc[combined.Entity.isna(), "Country"].unique())
combined = combined.drop(columns=["Country"])

Countries without standardisation: ['Germany Fed Rep' 'Yemen Arab Rep' 'Yemen P Dem Rep' 'Germany Dem Rep']


In [22]:
# Obtain all disasters figures
totals = combined.groupby(["Year", "Entity"], as_index=False).sum()
totals = totals.assign(disaster_type="All disasters")
# Obtain all countries figures
world = combined.groupby(["Year", "disaster_type"], as_index=False).sum()
world = world.assign(Entity="World")
# Add
combined = pd.concat([combined, world, totals])

In [23]:
# Add population
population = pd.read_csv("inputs/population.csv", encoding="latin1")
combined = combined.merge(population, on=["Entity", "Year"], how="left")

# Add rates
columns = ["deaths", "injured", "affected", "homeless", "total_affected"]
combined = combined.assign(
    **{f"{col}_rate_per_100k": combined[col]/combined["Population"]*100000 for col in columns}
)
combined = combined.drop(columns=["Population"])

In [24]:
# Add gdp
gdp = pd.read_csv("inputs/gdp.csv", encoding="latin1")
combined = combined.merge(gdp, on=["Entity", "Year"], how="left")

# Add rates
columns = ["total_damages"]
combined = combined.assign(
    **{f"{col}_pct_gdp": combined[col]/(combined["gdp"]/1000) * 100 for col in columns}
)
combined = combined.drop(columns=["gdp"])

In [25]:
disasters = pd.pivot_table(combined, columns='disaster_type', index=["Year", "Entity"])
mapping = {
    "All disasters": "all_disasters",
    "Drought": "drought",
    "Earthquake": "earthquake",
    "Extreme temperature": "temperature",
    "Flood": "flood",
    "Fog": "fog",
    "Glacial lake outburst": "glacial_lake",
    "Landslide": "landslide",
    "Mass movement (dry)": "mass_movement",
    "Storm": "storm",
    "Volcanic activity": "volcanic",
    "Wildfire": "wildfire",
}
disasters.columns = [f"{colname[0]}_{mapping[colname[1]]}" for colname in disasters.columns]
disasters = disasters.reset_index()

In [26]:
# Final processing
disasters = disasters[["Entity"] + [col for col in disasters.columns if col != "Entity"]]
disasters = disasters[disasters["Entity"].notna()]
disasters = disasters[(disasters.Year != 2021)]

In [27]:
# Export
disasters.to_csv("output/Natural disasters (EMDAT).csv", index=False)

In [28]:
# Decade and export
disasters = disasters.assign(Year=combined.Year//10*10)
disasters = disasters.groupby(["Entity", "Year"], as_index=False).mean()
disasters.to_csv("output/Natural disasters (EMDAT – decadal).csv", index=False)